In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')


In [2]:
import os
import json

# Data Wrangling
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import glob
import flammkuchen as fl
import shutil

import seaborn as sns
import matplotlib as mpl

### Custom functions

In [3]:
# from feature_extractuion_helpers import smooth_trace
# from megabouts_helper import labels_cat

from megabouts.utils import (
    bouts_category_name,
    bouts_category_name_short,
    bouts_category_color,
    cmp_bouts,
)

In [4]:
def flatten(l):
    return [item for sublist in l for item in sublist]

## pool data 

In [40]:
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\230307_visstim_2D") #rectangular arena # start from fish 1
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_round")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_2") #rectangular arena
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\13052024_visstim_2D_round")
master_path = Path(r"\\portulab.synology.me\data\Kata\Data\14052024_visstim_2D_round")

fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths, len(fish_paths)

([WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f0'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f1'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f2'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f3'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f4'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f5'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f6'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f7'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f8'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/14052024_visstim_2D_round/240514_f9')],
 10)

In [41]:
exp_name = Path(fish_paths[0]).parts[-2]
fish= 0
fish_id =  fish_paths[fish].name
exp_name = Path(fish_paths[fish]).parts[-2]
exp_name, fish_id

('14052024_visstim_2D_round', '240514_f0')

In [42]:
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\230307_visstim_2D_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_round_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_2_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\13052024_visstim_2D_round_")
out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\14052024_visstim_2D_round_")

### Tensor pooling

In [43]:
tensor_path = Path(r'\\portulab.synology.me\data\Kata\Processed_Data\tensor_data\tensors')

In [44]:
tensors_ = []

for ind, fish in enumerate(fish_paths):
    print ('Working on {}'.format(fish.name))
    fish_id = fish.name
    
    try:
        tensor = fl.load(Path(out_path/'{}_tail_tensor.h5'.format(fish_id)))
        print (tensor.shape)
        tensors_.append(tensor)
    except OSError:
        pass
    


Working on 240514_f0
Working on 240514_f1
(41, 50, 10)
Working on 240514_f2
(40, 50, 10)
Working on 240514_f3
(147, 50, 10)
Working on 240514_f4
(272, 50, 10)
Working on 240514_f5
(175, 50, 10)
Working on 240514_f6
(277, 50, 10)
Working on 240514_f7
(279, 50, 10)
Working on 240514_f8
(577, 50, 10)
Working on 240514_f9
(509, 50, 10)


In [45]:
### stack tensors per experiment
# Concatenate tensors along the first axis
big_tensor = np.concatenate(tensors_, axis=0)

# Output the shape of the resulting tensor
print(big_tensor.shape)
fl.save(tensor_path/ '{}_tensor_tail_full.h5'.format(exp_name), big_tensor)

(2317, 50, 10)


In [46]:
t_paths = list(tensor_path.glob('*_tensor_tail_full.h5*'))
t_paths, len(t_paths)
tensors_pooled =[]

for ind, path in enumerate(t_paths):
    print ('Working on {}'.format(path.name))
    try:
        tensor = fl.load(path)
        print (tensor.shape)
        tensors_pooled.append(tensor)
    except OSError:
        pass


Working on 13052024_visstim_2D_round_tensor_tail_full.h5
(2007, 50, 10)
Working on 14052024_visstim_2D_round_tensor_tail_full.h5
(2317, 50, 10)
Working on 22042024_visstim_2D_2_tensor_tail_full.h5
(1869, 50, 10)
Working on 22042024_visstim_2D_round_tensor_tail_full.h5
(9937, 50, 10)
Working on 230307_visstim_2D_tensor_tail_full.h5
(2676, 50, 10)


In [47]:
### stack tensors per experiment
# Concatenate tensors along the first axis
pooled_tensor = np.concatenate(tensors_pooled, axis=0)

# Output the shape of the resulting tensor
print(pooled_tensor.shape)
fl.save(tensor_path/ 'pooled_tensor_tail_full.h5', pooled_tensor)

(18806, 50, 10)
